网上看的思路理解以后自己写的(递归)：

首先 设一串数字字符串为 'abcd....'
首先观察'a'是不是0,如果是0,则不能decode, 所以返回0
如果不是0,那么decode('abcd...')的方法数量和 decode('bcd...')的方法数量相同,此外, 如果'ab'能组成一个两位数(即 10 -- 26 之间），那么还可以是 decode('abcd...') = decode('bcd..') + decode('cd...')

base case:
当数组减到 空以后，返回1

以此进行递归

In [6]:
def helper(string_):
    if string_=='':
        return 1
    else:
        if string_[0] == '0':
            return 0
        if 9<int(string_[:2])<=26:
            return helper(string_[1:])+helper(string_[2:])
        else:
            return helper(string_[1:])
        
def numDecodings(s):
    return helper(s)

In [7]:
numDecodings('112354622')

10

坏处：这种方法时间复杂度太高了，是O(2^n). 因为每算一个字符串，都要把之前的字符串全算一次。

优化：既然可以正向这样想，那么可以反过来看，还是'abcd', 那么decode('abcd') = decode('abc') + decode('ab')
类似地，如果d！=0, 那么可以单独组成一个数，至少 decode('abcd') = decode('abc')
此外，如果 cd 可以组成一个 大于等于10 小于等于26的两位数，那么 decode('abcd') = decode('abc') + decode('ab')

而decode('ab') 之类的之前已经计算过，所以从小到大，就可以放到一个数组里预先进行存储，然后进行读取，这样就不用重复计算了, 此时用到了动态规划的思路，所以，dp[i] = dp[i-1] + (看情况是否加dp[i-2])

此时也需要注意base case的问题，因为当i = 0 时，类似上面一种解法，应该是对应decode('')的情况，而不是对应decode('a')的情况。所以dp[0] = 1, 而dp[i+1] 对应的是字符串中第 i 个字符时的解法数量

In [8]:
def numDecoding(s):
    # 定义一个dp,用来存放计算好的数据
    dp = [0 for i in range(len(s)+1)]
    # 一开始, dp[0] = 1, 因为base case 是 '' 对应 1
    if s =='':
        return 0
    elif s[0] == '0':
        return 0
    dp[0] = 1
    for i in range(1, len(s)+1):
        if s[i-1] != '0':
            dp[i] += dp[i-1]
        ## 因为当i = 1时，i-2 = -1, 所以 dp[-1]此时不符合情况, 所以应该跳过i=1的时候
        if i != 1 and 10 <= int(s[i-2:i]) <= 26:
            dp[i]+=dp[i-2]
    return dp[i]

In [9]:
numDecoding('112354622')

10

实际上，一开始不用判断s是否以0开头. 就像我们递归时的动态判断一样：

In [11]:
def numDecoding(s):
    # 定义一个dp,用来存放计算好的数据
    dp = [0 for i in range(len(s)+1)]
    # 一开始, dp[0] = 1, 因为base case 是 '' 对应 1
    if s =='':
        return 0
    dp[0] = 1
    for i in range(1, len(s)+1): # 当两个条件都不满足时，只用+0就行了
        if s[i-1] != '0':
            dp[i] += dp[i-1]
        ## 因为当i = 1时，i-2 = -1, 所以 dp[-1]此时不符合情况, 所以应该跳过i=1的时候
        if i != 1 and 10 <= int(s[i-2:i]) <= 26:
            dp[i]+=dp[i-2]
    return dp[i]

In [12]:
numDecoding('112354622')

10

如果想要 dp[i] 对应字符串中第i位的解法, 那么需要对代码进行改进，需要针对i=1的情况特殊处理, 并且同时还要预先判断dp[0]的情况。因为此时dp[0]不再是对应的空字符串'', 而是字符串的第一位数

In [13]:
def numDecoding(s):
    # 定义一个dp,用来存放计算好的数据
    dp = [0 for i in range(len(s))]
    # 一开始, dp[0] = 1, 因为第一个字符只有一种解码方式（如果第一个字符是0的话那就直接return 0）
    if s =='':
        return 0
    elif s[0] == '0':
        return 0
    dp[0] = 1
    for i in range(len(s)):
        if s[i] != '0':
            dp[i] += dp[i-1]
        ## 因为当i = 1时，i-2 = -1, 所以 dp[-1]此时不符合情况, 所以应该跳过i=1的时候
        if i > 1 and 10 <= int(s[i-1:i+1]) <= 26:
            dp[i] += dp[i-2]
#        针对 i = 1 特殊情况处理：
        if i == 1 and len(s)>=2 and 10<=int(s[i-1:i+1])<=26:
            dp[i] += 1
    return dp[i]

In [14]:
numDecoding('112354622')

10

我们注意到，只有dp[i], dp[i-1], dp[i-2] 三个东西有用，那么我们可以只用一个长度为3的list来存储, 对于一个数组长度为 l, 则：
for i in range(len(L)):

 一开始 1 = 0
 
 i = 2: 2 = 1 + 0
 
 dp[i%3] = dp[(i-1)%3] + dp[(i-2)%3]
 
 i = 3: 然后 0 = 2 + 1
 
 dp[i%3] = dp[(i-1)%3] + dp[(i-2)%3]
 
 然后 1 = 0 + 2
 
 以此类推
 
 dp[i%3] = dp[(i-1)%3] + dp[(i-2)%3] 

In [15]:
def numDecodings(s):
    # dp[i] = dp[i-1] if s[i]!=0
    #               +dp[i-2] if "09"<s[i-1:i+1]<"27"
    if s == '':
        return 0
    dp = [0 for x in range(3)]
    dp[0] = 1
    for i in range(1, len(s)+1): #start from 1
        # update the position that will use
        dp[i%3]=0
        if s[i-1]!="0":
            dp[i%3] += dp[(i-1)%3]
        if i>1 and 10<=int(s[i-2:i]) <= 26:
            dp[i%3] += dp[(i-2)%3]
    return dp[len(s)%3]

In [16]:
numDecodings('112354622')

10

## 向大神学习 ---written by others

# 1

分析：

该方法首先，与上述只用长度为3的优化算法类似，巧用了 python内部运算的先后关系，用两个变量实现了 dp[i], dp[i-1], 和dp[i-2] 的作用

同时，固定了变量 dp1, dp2, 避免了要针对i=1进行处理的特殊情况

先理解如下表达：

In [17]:
a = 5;b = 4

a,b = b,a
print('a: ',a, 'b: ',b)

a = 5; b = 4
a = b
b = a

print('a: ',a, 'b: ',b)

a = 5; b = 4
t = b
b = a
a = t

print('a: ',a, 'b: ',b)

a:  4 b:  5
a:  4 b:  4
a:  4 b:  5


在上面的测试中，第一种方法通过 a,b = b,a 这样的先后运算关系，把第三种方法的三个变量缩减为2个

再来理解一下下面这个 两个if写在一行代表了什么

from -- https://stackoverflow.com/questions/14029245/putting-an-if-elif-else-statement-on-one-line

In [18]:
i=100
a = 1 if i<100 else 2 if i>100 else 0
print(a,'\n------------\n')

i=101
a = 1 if i<100 else 2 if i>100 else 0
print(a,'\n------------\n')

i=99
a = 1 if i<100 else 2 if i>100 else 0
print(a,'\n------------\n')

0 
------------

2 
------------

1 
------------



In [19]:
##通过观察，上面式子可以翻译为：
def c(i):
    if i<100:
        a =1 
    else:
        if i>100:
            a = 2
        else:
            a = 0
    return a

list(map(c,[100,101, 99]))

[0, 2, 1]

# 向大神学习1

In [20]:
def numDecodings(s):
    if s[0] == "0": return 0
    dp1 = dp2 = 1
    for i in range(1, len(s)):
        if s[i] == "0" and (s[i - 1] == "0" or s[i - 1] >= "3"): return 0
        dp1, dp2 = [dp2, dp1] if s[i] == "0" else [dp2, dp2 + dp1] if "10" <= s[i -1: i + 1] <= "26" else [dp2, dp2]
    return dp2

进行展开：

In [21]:
def numDecodings(s):
    if s[0] == "0": return 0
    dp1 =dp2 = 1
    for i in range(1, len(s)):
        # 判断会不会出现 00 或者 30 这样的情况。 如果有，则不可能存在解法，所以return 0
        if s[i] == "0" and (s[i - 1] not in ['1','2']): return 0
        # 判断之后，判断s[i]是否等于0，如果是, 则s[i-1]只能单独和s[i]组成一个组合，所以此时dp[i] = dp[i-2].
        # 如果s[i] 不等于0， 那么dp[i] = dp[i-1] 至少，如果能和前面的数组成 两位数，则再加 dp[i-2]
        # 事实上，整个流程只牵扯了 dp[i-1]和dp[i-2], 而dp[i]只是一个存储 dp[i-1]和dp[i-2]的容器而已
        
        #########翻译写法1##########
        # 在这里 dp2作用为更新前的dp[i-1], 更新时的dp[i];
        # dp1作用为更新前的dp[i-2]和更新时的dp[i-1]
        # 当s[i] == '0'时，此时s[i]只能和前面组成组合，所以此时更新：dp[i] = dp[i-2]
        if s[i] == "0":
#             dp1, dp2 = dp2, dp1
            new_dpi = dp2 # dp[i] = dp[i-2]
            dp2 = dp1 # update dp[i-1]
        # now the dp2 becomes the dp[i-1] in the future
            dp1 = new_dpi # update the dp[i-2] 
        else:
            # 当 可以组成两位数时:
            # 等式右边: dp2 = dp[i-1], dp1 = dp[i-2]
            # 等式左边: dp2 = dp[i], dp1 = dp[i-1]
            if "10"<=s[i-1:i+1]<="26":
                #dp1, dp2 = dp2, dp2 + dp1
                new_dpi  = dp2 + dp1 # dp[i] = dp[i-1] + dp[i-2]
                dp1 = dp2 # update dp[i-2]
                dp2 = new_dpi #update dp[i-1]
            else:
            # 当 不能组成两位数时，dp[i] = dp[i-1], 然后进行更新，即，dp[i-2] = dp[i-1]; dp[i-1] = dp[i]
            # 所以，
                #dp1, dp2 = dp2, dp2
                new_dpi = dp2 #dp[i] = dp[i-1]
                dp1 = dp2 # update dp[i-2]
                dp2 = new_dpi # update dp[i-1]
    return dp2
        
        
        
        
        
        
        
        
        
        
        
        
        #########翻译写法2###########
        
        
        # 在本例中, dp2相当于dp[i-2]以及同时充当dp[i]的作用；而dp1 相当于dp[i-1]的作用。
        # 为了防止 dp2 在被dp1 更新的时候，失去了代表dp[i-2]的作用，所以需要先判断是否需要加 dp[i-2]
        # 即需要先判断 s[i-1:i+1] 时候，是否能组成 10 至 26 之间的数，来判断是否需要先让dp[i] = dp[i-2]
        # 然后，再判断s[i]是否为0进行独立, 即 此时dp[i]再加dp[i-1]
        # 也就是 先判断dp2变不变，如果不变，看看是否需要加dp1。
#         if '10'<=s[i-1:i+1]<='26':
#             # dp[i] = dp[i-1]＋dp[i-2]
#             dp2 = dp2 + dp1
#         else:
#             if s[i] != '0':
#                 # dp[i] = dp[i-1]
#                 dp2 = dp1
#         # update i = i+1, 所以 d[i]变为d[i-1], d[i-1]变为d[i-2]
#         dp1, dp2 = dp2, dp1
#     return dp1

In [22]:
numDecodings('112354622')

10

# 向大神学习2：

In [23]:
def numDecodings(s):
    # Initialization, 
    # v =0 , w is 1 or 0, p = ''
    v, w, p = 0, int(s>''), ''
    for d in s:
        # for v, the previous number of ways is equal to the ''previous number of ways', which is keep updating # dp[i-1]
        # for w, if the current digit is larger than zero, and the  current digit plus previous  
        # p is the previous number in the current state, which is updating
        
        # 注意等式右边，w是dp[i-1], 也就是上一位数解的方法数量，v是dp[i-2]上上步的解题数量，p是s[i-1], d是s[i]
        v, w, p = w, (d>'0')*w + (9<int(p+d)<27)*v, d
    return w

In [24]:
numDecodings('112354622')

10

reference：

大神 2：
https://leetcode.com/problems/decode-ways/discuss/30379/1-liner-O(1)-space

大神1：（出于该帖子评论处）
https://leetcode.com/problems/decode-ways/discuss/30352/Accpeted-Python-DP-solution 


动态规划灵感：
https://www.youtube.com/watch?v=qli-JCrSwuk

